In [1]:
import numpy as np
import math
import pandas as pd
import random
import time

**转换为邻接表**

In [2]:
graph = {}
f = open('ca-AstroPh.txt', 'r')
t1 = time.perf_counter()
while True:
    try:
        line = f.readline().strip().split(' ')  # 去掉回车并分割
        i = int(line[0])
        j = int(line[1])
        try:
            graph[i].add(j)
        except Exception:
            graph[i] = set([])
            graph[i].add(j)
        try:
            graph[j].add(i)
        except Exception:
            graph[j] = set([])
            graph[j].add(i)
    except ValueError:
        break
t2 = time.perf_counter()
t = (t2 - t1) * 1000  # 输出单位为毫秒
print(t)
f.close()
for i in range(1, 18772):
    graph[i] = list(graph[i])

366.3696999999999


**随机重排**

**在已经建立好的邻接表中随机选择一个节点作为最小哈希签名，对每个节点重复多次（生成随机数）**

In [3]:
# 最小哈希签名矩阵，8000 * 18771
t3 = time.perf_counter()
min_hash_sig = [[[0] for _ in range(18771)] for _ in range(8000)]
for i in range(8000):
    for j in range(18771):
        random.shuffle(graph[j + 1])
        min_hash_sig[i][j] = graph[j + 1][0]

t4 = time.perf_counter()
t = (t4 - t3) * 1000  # 输出单位为毫秒
print(t)
# min_hash_sig_matrix = pd.DataFrame(data = min_hash_sig, index = None, columns = None)
# min_hash_sig_matrix.to_csv('min_hash_sig.csv')

1379262.8312


**分桶**

In [4]:
bucket = {}
t5 = time.perf_counter()
for i in range(4000):
    for j in range(18771):
        sig = str(min_hash_sig[i * 2][j]) + str(min_hash_sig[i * 2 + 1][j])
        try:
            bucket[sig].add(j + 1)
        except Exception:
            bucket[sig] = set([])
            bucket[sig].add(j + 1)

t6 = time.perf_counter()
t = (t6 - t5) * 1000  # 输出单位为毫秒
print(t)
print(len(bucket))

78710.86739999987
4593482


**待查询节点编号为326时**

In [5]:
k = 326
t7 = time.perf_counter()
# 将对应桶合并
bucket_k = set([])
for i in range(4000):
    sig = str(min_hash_sig[i * 2][k - 1]) + str(min_hash_sig[i * 2 + 1][k - 1])
    bucket_k = bucket_k.union(bucket[sig])
# print(bucket_k)
t8 = time.perf_counter()
t = (t8 - t7) * 1000  # 输出单位为毫秒
print(t)

85.54380000009587


In [6]:
# 计算Jaccard相似度并排序
rank = {}
bucket_k = list(bucket_k)
bucket_k.remove(k)
t9 = time.perf_counter()
for item in bucket_k:
    intersect = list(set(graph[k]) & set(graph[item]))
    union = list(set(graph[k]) | set(graph[item]))
    rank[item] = (len(intersect) / len(union))

if len(bucket_k) >= 10:
    l = 0
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])
        l += 1
        if l == 10:
            break
else:
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])

t10 = time.perf_counter()
t = (t10 - t9) * 1000  # 输出单位为毫秒
print(t)

285 0.6692160611854685
2295 0.35988200589970504
26 0.30976430976430974
2489 0.29693486590038315
2277 0.2954898911353033
2288 0.2613065326633166
350 0.2564102564102564
4395 0.2548262548262548
304 0.24864864864864866
126 0.24572317262830481
78.0369999999948


In [8]:
# 验证准确性
k = 326
rank = {}
for i in range(18771):
    if i + 1 == k:
        continue
    intersect = list(set(graph[k]) & set(graph[i + 1]))
    union = list(set(graph[k]) | set(graph[i + 1]))
    rank[i + 1] = (len(intersect) / len(union))

l = 0
for item in sorted(rank, key = rank.__getitem__, reverse = True):
    print(item, rank[item])
    l += 1
    if l == 10:
        break

285 0.6692160611854685
2295 0.35988200589970504
26 0.30976430976430974
2489 0.29693486590038315
2277 0.2954898911353033
2288 0.2613065326633166
350 0.2564102564102564
4395 0.2548262548262548
304 0.24864864864864866
126 0.24572317262830481


**待查询节点编号为980时**

In [9]:
k = 980
t7 = time.perf_counter()
# 将对应桶合并
bucket_k = set([])
for i in range(4000):
    sig = str(min_hash_sig[i * 2][k - 1]) + str(min_hash_sig[i * 2 + 1][k - 1])
    bucket_k = bucket_k.union(bucket[sig])
# print(bucket_k)
t8 = time.perf_counter()
t = (t8 - t7) * 1000  # 输出单位为毫秒
print(t)

3923.2629999969504


In [10]:
# 计算Jaccard相似度并排序
rank = {}
bucket_k = list(bucket_k)
bucket_k.remove(k)
t9 = time.perf_counter()
for item in bucket_k:
    intersect = list(set(graph[k]) & set(graph[item]))
    union = list(set(graph[k]) | set(graph[item]))
    rank[item] = (len(intersect) / len(union))

if len(bucket_k) >= 10:
    l = 0
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])
        l += 1
        if l == 10:
            break
else:
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])

t10 = time.perf_counter()
t = (t10 - t9) * 1000  # 输出单位为毫秒
print(t)

1141 0.8148148148148148
897 0.7777777777777778
911 0.7307692307692307
959 0.7307692307692307
940 0.7241379310344828
903 0.7037037037037037
993 0.7037037037037037
990 0.6923076923076923
898 0.6875
945 0.6470588235294118
50.93439999836846


In [11]:
# 验证准确性
k = 980
rank = {}
for i in range(18771):
    if i + 1 == k:
        continue
    intersect = list(set(graph[k]) & set(graph[i + 1]))
    union = list(set(graph[k]) | set(graph[i + 1]))
    rank[i + 1] = (len(intersect) / len(union))

l = 0
for item in sorted(rank, key = rank.__getitem__, reverse = True):
    print(item, rank[item])
    l += 1
    if l == 10:
        break

1141 0.8148148148148148
897 0.7777777777777778
911 0.7307692307692307
959 0.7307692307692307
940 0.7241379310344828
903 0.7037037037037037
993 0.7037037037037037
990 0.6923076923076923
898 0.6875
945 0.6470588235294118


**待查询节点编号为5000时**

In [12]:
k = 5000
t7 = time.perf_counter()
# 将对应桶合并
bucket_k = set([])
for i in range(4000):
    sig = str(min_hash_sig[i * 2][k - 1]) + str(min_hash_sig[i * 2 + 1][k - 1])
    bucket_k = bucket_k.union(bucket[sig])
# print(bucket_k)
t8 = time.perf_counter()
t = (t8 - t7) * 1000  # 输出单位为毫秒
print(t)

253.1395000005432


In [13]:
# 计算Jaccard相似度并排序
rank = {}
bucket_k = list(bucket_k)
bucket_k.remove(k)
t9 = time.perf_counter()
for item in bucket_k:
    intersect = list(set(graph[k]) & set(graph[item]))
    union = list(set(graph[k]) | set(graph[item]))
    rank[item] = (len(intersect) / len(union))

if len(bucket_k) >= 10:
    l = 0
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])
        l += 1
        if l == 10:
            break
else:
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])

t10 = time.perf_counter()
t = (t10 - t9) * 1000  # 输出单位为毫秒
print(t)

5015 0.48148148148148145
4994 0.4642857142857143
5017 0.4482758620689655
6672 0.4444444444444444
4993 0.40625
6674 0.35
5011 0.3333333333333333
4998 0.3170731707317073
5009 0.2765957446808511
4978 0.2727272727272727
5.159399999683956


In [14]:
# 验证准确性
k = 5000
rank = {}
for i in range(18771):
    if i + 1 == k:
        continue
    intersect = list(set(graph[k]) & set(graph[i + 1]))
    union = list(set(graph[k]) | set(graph[i + 1]))
    rank[i + 1] = (len(intersect) / len(union))

l = 0
for item in sorted(rank, key = rank.__getitem__, reverse = True):
    print(item, rank[item])
    l += 1
    if l == 10:
        break

5015 0.48148148148148145
4994 0.4642857142857143
5017 0.4482758620689655
6672 0.4444444444444444
4993 0.40625
6674 0.35
5011 0.3333333333333333
4998 0.3170731707317073
5009 0.2765957446808511
4978 0.2727272727272727


**待查询节点编号为18000时**

In [15]:
k = 18000
t7 = time.perf_counter()
# 将对应桶合并
bucket_k = set([])
for i in range(4000):
    sig = str(min_hash_sig[i * 2][k - 1]) + str(min_hash_sig[i * 2 + 1][k - 1])
    bucket_k = bucket_k.union(bucket[sig])
# print(bucket_k)
t8 = time.perf_counter()
t = (t8 - t7) * 1000  # 输出单位为毫秒
print(t)

1409.9779000025592


In [16]:
# 计算Jaccard相似度并排序
rank = {}
bucket_k = list(bucket_k)
bucket_k.remove(k)
t9 = time.perf_counter()
for item in bucket_k:
    intersect = list(set(graph[k]) & set(graph[item]))
    union = list(set(graph[k]) | set(graph[item]))
    rank[item] = (len(intersect) / len(union))

if len(bucket_k) >= 10:
    l = 0
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])
        l += 1
        if l == 10:
            break
else:
    for item in sorted(rank, key = rank.__getitem__, reverse = True):
        print(item, rank[item])

t10 = time.perf_counter()
t = (t10 - t9) * 1000  # 输出单位为毫秒
print(t)

18001 0.5
17319 0.09090909090909091
0.3849000022455584


In [17]:
# 验证准确性
k = 18000
rank = {}
for i in range(18771):
    if i + 1 == k:
        continue
    intersect = list(set(graph[k]) & set(graph[i + 1]))
    union = list(set(graph[k]) | set(graph[i + 1]))
    rank[i + 1] = (len(intersect) / len(union))

l = 0
for item in sorted(rank, key = rank.__getitem__, reverse = True):
    print(item, rank[item])
    l += 1
    if l == 10:
        break

18001 0.5
17319 0.09090909090909091
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
